In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 

from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

from simpletransformers.language_modeling import LanguageModelingModel

import spacy

from sklearn.svm import LinearSVC
from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import itertools
from collections import Counter

from tqdm import tqdm
import torch

import networkx as nx

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from functools import partial

import pickle

from collections import deque

from torchvision import models
import torch.nn.functional as F
from torch import nn

from sklearn.utils import resample

import fasttext


from typing import List
from pathlib import Path
from joblib import dump, load

import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/home/roshansk/YelpAnalysis/')
from utils import *



/home/roshansk/Covid_Bert/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/roshansk/Covid_Bert/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/roshansk/Covid_Bert/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/roshansk/Covid_Bert/env/lib/python3.6/site-packages/ten

In [2]:
nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl')
# nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl_subword/')

sys.path.insert(0,'/data2/Datasets/')
from preprocess import *

dataFolder = '/data2/Datasets/Raw'

In [59]:
dataFolder = '/data2/Datasets/Raw'

## Dataset that will be used for creating the lexicon
lexiconDataset = 'nrc_joy'
# dataList = ['nrc_joy','song_joy','dialog_joy','friends_joy']

##################################



trainDf, devDf, testDf = splitData(getData(dataFolder, lexiconDataset))

In [61]:
wordCount = 0
for i in range(len(trainDf)):
    if 'rumors' in trainDf.iloc[i]['text']:
        wordCount += 1
    

In [62]:
wordCount

5

In [38]:
trainDf.label.value_counts()

1    5952
0    5952
Name: label, dtype: int64

In [36]:
len(testDf)

1502

### Load Data & Model Training

In [5]:
trainDf, devDf, testDf = splitData(getData(dataFolder, 'empathy'))

trainData = generateFastTextData_Spacy(trainDf, nlp, textVariable = 'text')

testData = generateFastTextData_Spacy(testDf, nlp, textVariable = 'text')

model = trainSVM(trainData, testData, trainDf, testDf)

lexicon = generateLexicon_SVM(model,trainDf, nlp)

In [24]:
lexicon.head()

,word,score,wordCount,SVM_Rank
5346,circuses,41.932019,1,0
7783,graham,32.510310,1,1
4070,fernandez,32.277996,2,2
3278,shipwreck,29.755081,2,3
7312,job.i,26.816461,1,4


In [25]:
lexiconWords, lexiconMap = getLexicon(df = lexicon)


### Experiments

In [12]:
dataList = ['dialog','amazon_finefood_subset','amazon_toys_subset','yelp_subset','empathy','nrc_joy']
for data in dataList:
    results = testSVM(model, data,lexiconWords, lexiconMap, nlp, dataFolder)
    

dialog , 0.838 , 0.908 , 0.838 , 0.907
amazon_finefood_subset , 0.869 , 0.928 , 0.872 , 0.929
amazon_toys_subset , 0.94 , 0.969 , 0.935 , 0.966
yelp_subset , 0.884 , 0.924 , 0.879 , 0.921
empathy , 0.505 , 0.614 , 0.548 , 0.621
nrc_joy , 0.635 , 0.366 , 0.629 , 0.333


In [26]:
dataList = ['songs','emobank']
for data in dataList:
    results = testSVM(model, data,lexiconWords, lexiconMap, nlp, dataFolder)

songs , 0.7744322344322344 , 0.8711197856579798 , 0.777 , 0.874
emobank , 0.37616689332460285 , 0.5391576140313964 , 0.626 , 0.013


### Lexicon Cleaning

In [3]:
import enchant
d = enchant.Dict("en_US")

In [7]:
df = pd.read_csv("Lexicons/amazon_finefood_subset_lexicon.csv")
df['inDict'] = df.word.apply(lambda x : d.check(x))

In [10]:
df.head()

,word,score,wordCount,SVM_Rank,inDict
0,igourmet.com,49.484859,5,0,False
1,7-pound,45.030954,1,1,True
2,omega-3s,39.806286,1,2,False
3,lingonberries,39.514822,11,3,False
4,whonu,39.031988,4,4,False


In [11]:
df[df.inDict==True].head(10)

,word,score,wordCount,SVM_Rank,inDict
1,7-pound,45.030954,1,1,True
8,10-star,34.937306,1,8,True
9,40-pound,34.309356,1,9,True
10,25-pound,33.788460,1,10,True
12,relaxed,33.576705,7,12,True
13,15-pound,33.362140,9,13,True
15,15-ounce,33.290025,2,15,True
17,fastest,32.692416,1,17,True
18,vinaigrettes,32.274410,1,18,True
21,marinades,31.725041,5,21,True


In [12]:
df[df.inDict==False].head(10)

,word,score,wordCount,SVM_Rank,inDict
0,igourmet.com,49.484859,5,0,False
2,omega-3s,39.806286,1,2,False
3,lingonberries,39.514822,11,3,False
4,whonu,39.031988,4,4,False
5,cloudberries,38.337925,2,5,False
6,versital,38.187562,2,6,False
7,crispbreads,35.062756,4,7,False
11,igourmet,33.687246,2,11,False
14,versitile,33.342217,2,14,False
16,tzatziki,32.797057,4,16,False


#### PyDictionary

In [19]:
from PyDictionary import PyDictionary
dictionary=PyDictionary()

In [22]:
dictionary.meaning('sdfjs')

Error: The Following Error occured: list index out of range


#### NLTK

In [14]:
from nltk.corpus import words

In [45]:
df = pd.read_csv("Lexicons/nrc_joy_lexicon.csv")
d = set(words.words())
df['inDict'] = df.word.apply(lambda x : x in d )

In [46]:
df[df.inDict==True].head(10)

,word,score,wordCount,SVM_Rank,inDict
3,sunshiny,11.012002,1,3,True
7,meridian,10.243682,1,7,True
9,yule,10.213697,2,9,True
10,gladness,10.135745,5,10,True
11,joy,10.134706,388,11,True
13,jolly,9.946622,5,13,True
14,fettling,9.884922,2,14,True
17,radiate,9.732559,5,17,True
20,hosanna,9.437616,1,20,True
23,joyous,9.208967,15,23,True


In [47]:
df[df.inDict==True].sort_values('SVM_Rank',ascending=False).head(10)

,word,score,wordCount,SVM_Rank,inDict
23335,afraid,-9.844687,333,23335,True
23329,disrespect,-9.209575,1,23329,True
23328,backfire,-9.079291,1,23328,True
23326,spoiler,-8.827825,1,23326,True
23324,hurley,-8.750281,1,23324,True
23320,disapproval,-8.544347,1,23320,True
23318,unauthorized,-8.391131,2,23318,True
23317,silenced,-8.289362,1,23317,True
23314,feared,-8.127090,1,23314,True
23313,abusive,-8.126354,1,23313,True


In [48]:
df[df.inDict==False].head(10)

,word,score,wordCount,SVM_Rank,inDict
0,joy-,15.364728,1,0,False
1,lemsip,12.229379,3,1,False
2,carols,11.545457,14,2,False
4,joys,10.841129,9,4,False
5,sunshines,10.728400,1,5,False
6,joyness,10.272427,1,6,False
8,achey,10.220822,1,8,False
12,sunbeams,10.044808,2,12,False
15,excelsis,9.840990,1,15,False
16,unltd,9.833132,1,16,False


In [58]:
df[df.word=='rumors']

,word,score,wordCount,SVM_Rank,inDict
22130,rumors,-3.78979,5,22130,False


In [50]:
df.sort_values('wordCount',ascending=False).head(50)

,word,score,wordCount,SVM_Rank,inDict
6114,.,0.553127,11346,6114,False
10001,i,-0.970493,10187,10001,True
6167,the,0.529517,8736,6167,True
6644,!,0.340895,7816,6644,False
5085,to,0.986616,7797,5085,True
7591,",",-0.028598,6048,7591,False
8405,a,-0.372088,5629,8405,True
8251,#,-0.305723,5539,8251,False
5577,my,0.774387,5236,5577,True
3679,and,1.673186,5187,3679,True
